# GPU Mining Rig vs. Buying Crypto

Analysis on whether holding a mining rig is more profitable than just buying the crypto asset. <br>
1. Calculate the cost of mining rig. Price and Hashing Power<br>
2. Calculate the over time how much profit the rig can make<br>
    2.1 Calculate historical mining profitability<br>
    2.

In [ ]:
rig_hash_power = 100

In [64]:
#Clean eth_mining_probitability.txt


%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import yfinance as yf
from yahoofinancials import YahooFinancials
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import json
from io import StringIO
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:

stri = open('data/eth_mining_profitability.txt', 'r').read()

stri = stri.replace('new Date("',"")
stri = stri.replace('")',"")
stri = stri.replace("],[", "\n")
stri = stri.replace("[", "")
stri = stri.replace("]", "")


In [4]:

df = pd.read_csv(StringIO(stri), index_col=0, header=None)
df = df.rename(columns={1 : "USD per MH"})
df.index.names = ['Date']
df.index = pd.to_datetime(df.index)


In [5]:
eth_min_prof_df = df

In [6]:
eth_min_prof_df

,USD per MH
Date,
2015-07-30,9.3550
2015-07-31,2.1920
2015-08-01,1.4420
2015-08-02,1.0220
2015-08-03,0.9310
...,...
2021-01-30,0.0866
2021-01-31,0.0903
2021-02-01,0.0934


In [7]:
#drop 2015
eth_min_prof_df = eth_min_prof_df.drop(eth_min_prof_df.loc['2015'].index)

In [8]:


eth_min_prof_df['Day Profit'] = eth_min_prof_df['USD per MH'] * rig_hash_power
#eth_min_prof_df['ETH Profit'] = 

In [9]:
eth_min_prof_df

,USD per MH,Day Profit
Date,,
2016-01-01,0.0476,4.76
2016-01-02,0.0501,5.01
2016-01-03,0.0498,4.98
2016-01-04,0.0493,4.93
2016-01-05,0.0496,4.96
...,...,...
2021-01-30,0.0866,8.66
2021-01-31,0.0903,9.03
2021-02-01,0.0934,9.34


In [10]:
#get prices for period

yf_startDate = eth_min_prof_df.index[0].strftime("%Y-%m-%d")
yf_endDate = eth_min_prof_df.index[-1].strftime("%Y-%m-%d")
crypto_df = yf.download('ETH-USD', 
                      start=yf_startDate, 
                      end=yf_endDate,
                        auto_adjust=True)


crypto_df = crypto_df.drop(columns=['Open', 'Volume', 'High', 'Low'])
crypto_df.columns = ['ETH-USD']

[*********************100%***********************]  1 of 1 completed


In [11]:
crypto_df.shift(0)

,ETH-USD
Date,
2015-12-31,0.933542
2016-01-01,0.948024
2016-01-02,0.937124
2016-01-03,0.971905
2016-01-04,0.954480
...,...
2021-01-29,1382.522827
2021-01-30,1376.115479
2021-01-31,1314.986206


In [45]:
eth_min_prof_df['ETH per MH'] = eth_min_prof_df['USD per MH'] / crypto_df['ETH-USD']
eth_min_prof_df['ETH Mined'] = eth_min_prof_df['ETH per MH'] * rig_hash_power

In [46]:
subset_df = eth_min_prof_df.dropna()
subset_df = subset_df.loc['2017-07-01' :]

In [47]:
ifconvertdailyUSD = eth_min_prof_df.loc[yf_startDate :]['Day Profit'].sum()
total_eth_mined = subset_df['ETH Mined'].sum()
total_ethusd_mined = total_eth_mined * crypto_df.tail(1)
total_ethusd_mined = total_ethusd_mined.iloc[0]['ETH-USD']



In [48]:
print("Total ETH Mined = " + str(total_eth_mined.round(2)))
print("USD Equivalent (" + str(crypto_df.index[-1].strftime("%Y-%m-%d")) + ") = " + str(total_ethusd_mined.round(2)))

Total ETH Mined = 13.38
USD Equivalent (2021-02-02) = 20266.2


In [49]:
amt_buy_eth = 2000 / crypto_df.loc['2017-07-01']['ETH-USD']

In [50]:
print("Amount of ETH for US$2000 on '2017-07-01' = " + str(amt_buy_eth.round(2)))
amt_buy_eth_hold_USD = amt_buy_eth * crypto_df.tail(1)
amt_buy_eth_hold_USD = amt_buy_eth_hold_USD.iloc[0]['ETH-USD']
print("USD Equivalent (" + str(crypto_df.index[-1].strftime("%Y-%m-%d")) + ") = " + str(amt_buy_eth_hold_USD.round(2)))

Amount of ETH for US$2000 on '2017-07-01' = 7.28
USD Equivalent (2021-02-02) = 11035.6


In [53]:
subset_df
subset_df['Day Profit (Cum)'] = subset_df['Day Profit'].cumsum()

In [69]:

#subset_df['Day Profit (Cum)'].plot(color='r')

In [70]:
breakeven_df = subset_df.loc[subset_df['Day Profit (Cum)'] > 2000]

In [71]:
breakeven_df

,USD per MH,Day Profit,ETH per MH,ETH Mined,Day Profit (Cum)
Date,,,,,
2018-03-21,0.0448,4.48,0.000080,0.007975,2000.890
2018-03-22,0.0436,4.36,0.000081,0.008079,2005.250
2018-03-23,0.0410,4.10,0.000076,0.007598,2009.350
2018-03-24,0.0428,4.28,0.000081,0.008130,2013.630
2018-03-25,0.0409,4.09,0.000078,0.007801,2017.720
...,...,...,...,...,...
2021-01-29,0.0955,9.55,0.000069,0.006908,4547.964
2021-01-30,0.0866,8.66,0.000063,0.006293,4556.624
2021-01-31,0.0903,9.03,0.000069,0.006867,4565.654
